In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('C:\\Users\\gabri\\Documents\\MESTRADO_OBITO_FETAL_COVID\\dados_sim_dofet_select.csv',sep=';')
# Selecionar def_obito_parto
# Anos de interesse
df = df[df['ano_obito'].isin([2019, 2020])]
df = df.reset_index(drop=True)
df['ano_obito'] = [str(i) for i in df['ano_obito']]
df['TIPOBITO'] = [str(i) for i in df['TIPOBITO']]

In [ ]:
df.info()

In [ ]:
def data_quality(dataset, chave=None):
    # 1. Tipos de Variáveis
    tipos_variaveis = dataset.dtypes

    # 2. Frequência das Variáveis Categóricas
    #frequencia_categoricas = dataset.select_dtypes(include='object').apply(pd.Series.value_counts)

    # 3. Valores Discrepantes (Numéricas)
    def detectar_outliers(col):
        q1 = col.quantile(0.25)
        q3 = col.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = col[(col < lower_bound) | (col > upper_bound)]
        return outliers

    outliers_numericos = dataset.select_dtypes(include=np.number).apply(detectar_outliers)

    # 4. Estatísticas básicas para variáveis numéricas
    estatisticas_numericas = dataset.describe()

    # 5. Pontos Faltantes
    porcentagem_faltantes = dataset.isnull().mean() * 100

    # 6. Dados Duplicados
    if chave is not None:
        duplicados = dataset.duplicated(subset=chave).sum()
    else:
        duplicados = dataset.duplicated().sum()

    # Armazenar os resultados em um dicionário
    resultados = {
        "tipos_variaveis": tipos_variaveis,
        #"frequencia_categoricas": frequencia_categoricas,
        "outliers_numericos": outliers_numericos,
        "estatisticas_numericas": estatisticas_numericas,
        "porcentagem_faltantes": porcentagem_faltantes,
        "duplicados": duplicados
    }

    return resultados

# Exemplo de uso da função:
# Supondo que 'dataset' seja o seu DataFrame
# Para procurar duplicados por uma chave específica, como por exemplo 'ID':
# result = data_quality(dataset, chave='ID')
# Para procurar duplicados pela linha inteira:
# result = data_quality(dataset)
# print(result)

In [ ]:
aa=data_quality(dataset=df, chave=None)

In [ ]:
# Dados Duplicados
duplicados = aa["duplicados"]

print(f"""
Como não temos uma variavel identificadora esses valores podem ser reais, tendo em vista a baixa quantidade {round(duplicados/len(df),5)*100} % da base
, não será excluído.

""")

In [ ]:
# Estatisticas das variaveis nu méricas
aa[ "estatisticas_numericas"]
# Remover peso zero
# Ou Input 
# Remover Idade acima de X
# Fazer um corte aproximadamente 6 desvios da média 

In [ ]:
# Outliers
bb=aa["outliers_numericos"]
bb[~pd.isna(bb['IDADEMAE'])]

In [ ]:
aa["porcentagem_faltantes"]
# Remover  pontos faltantes de idade e peso
# Utilizar o dado 'cru' e depois limpo na descritiva apontando as diferenças.

In [ ]:
df[df['IDADEMAE'].isna()]

In [ ]:
df[df['PESO'].isna()]

In [ ]:
# Verificar contagem de valores únicos e valores ausentes para cada variável categórica
for col in df.select_dtypes(include=['object']):
    print(f'\nContagem de valores únicos para {col}:')
    print(df[col].value_counts())
    print(f'\nValores ausentes para {col}: {df[col].isnull().sum()}')

# Região pode ser um caminho.
# Olhar os bancos de nascimentos para verificar municípios com poucos partos/mês.
# Ignorado Descartar

# Categorizar PESO (Pequeno, adequado e (Gig 1 e 2(Macrossômico))) Função padrão, lembrar de pedir!
# Variavel X combinado da iadade gestacional com o peso (Gig 1 e 2(Macrossômico))
# Manter o PESO em variável absoluta.


In [ ]:
categoricas = [
    'ocor_MUNNOMEX', 'ocor_CAPITAL', 'ocor_SIGLA_UF',
       'ocor_REGIAO', 'def_escol_mae', 'def_raca_cor', 'def_sexo',
      'def_parto', 'def_gravidez', 'def_gestacao', 'ano_obito'
]
for i in categoricas:
    if i != 'ano_obito':
        print(f'Variavel:{i} \n {df[["ano_obito", i]].value_counts(normalize=True) * 100}')
    else:
        print(f'Variavel:{i} \n {df[i].value_counts(normalize=True) * 100}')
    
# Variavel raça com pouco preenchimento

In [ ]:
# Nas variaveis peso e idade da mae farei o input de dados faltante pela mediana.

mediana_idade = np.nanmedian(df['IDADEMAE'])
df['IDADEMAE'] = np.where(df['IDADEMAE'].isna(), mediana_idade, df['IDADEMAE'])


mediana_peso = np.nanmedian(df['PESO'])
df['PESO'] = np.where(df['PESO'].isna(), mediana_peso, df['PESO'])

In [ ]:
 np.nanmedian(df['IDADEMAE'])

In [ ]:
# Removendo os outliers de idade
df = df[df['IDADEMAE'] < 50]

In [ ]:
idade_2019 = df[df['ano_obito']=='2019']['IDADEMAE']
idade_2020 = df[df['ano_obito']=='2020']['IDADEMAE']
 
# plotting first histogram
plt.hist(idade_2019)
 
# plotting second histogram
plt.hist(idade_2020)
 
# Showing the plot using plt.show()
plt.show()

In [ ]:
peso_2019 = df[df['ano_obito']=='2019']['PESO']
peso_2020 = df[df['ano_obito']=='2020']['PESO']
 
# plotting first histogram
plt.hist(peso_2019)
 
# plotting second histogram
plt.hist(peso_2020)
 
# Showing the plot using plt.show()
plt.show()

In [ ]:
# Verificar a correlação entre variáveis categóricas e a variável alvo (TIPOBITO)
# Isso pode ser feito usando tabelas de contingência ou gráficos de barras empilhadas, dependendo do número de categorias
categoricas = [
    'ocor_MUNNOMEX', 'ocor_CAPITAL', 'ocor_SIGLA_UF',
       'ocor_REGIAO', 'def_escol_mae', 'def_raca_cor', 'def_sexo',
      'def_parto', 'def_gravidez', 'def_gestacao'
]
# Por exemplo:
for i in categoricas:
    contingency_table = pd.crosstab(index=df[i], columns=df['ano_obito'], normalize='index')
    print(f'Variável: {i} \n\n{contingency_table}')

In [ ]:
df.to_csv('base_tratada.csv', index=False)